# Coverage of fragments in STARR-seq results (Input 20x)
- depth in bedgraph format
- depth per base

**Set environment**

In [2]:
source ../config_duke.sh -v

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_STARRseq/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect



## Get coverage of all chromosomes

**Genome file**

In [3]:
FD_GEN=${FD_ANN}/genome/hg38
ls ${FD_GEN}

fasta                   hg38.chrom.sizes.chr16  hg38.chrom.sizes.chr4
hg38.chrom.sizes        hg38.chrom.sizes.chr17  hg38.chrom.sizes.chr5
hg38.chrom.sizes.chr1   hg38.chrom.sizes.chr18  hg38.chrom.sizes.chr6
hg38.chrom.sizes.chr10  hg38.chrom.sizes.chr19  hg38.chrom.sizes.chr7
hg38.chrom.sizes.chr11  hg38.chrom.sizes.chr2   hg38.chrom.sizes.chr8
hg38.chrom.sizes.chr12  hg38.chrom.sizes.chr20  hg38.chrom.sizes.chr9
hg38.chrom.sizes.chr13  hg38.chrom.sizes.chr21  hg38.chrom.sizes.chrX
hg38.chrom.sizes.chr14  hg38.chrom.sizes.chr22  hg38.chrom.sizes.chrY
hg38.chrom.sizes.chr15  hg38.chrom.sizes.chr3   hg38.chrom.sizes.rm


In [4]:
FD_GEN=${FD_ANN}/genome/hg38
cat ${FD_GEN}/hg38.chrom.sizes

chr1	248956422
chr2	242193529
chr3	198295559
chr4	190214555
chr5	181538259
chr6	170805979
chr7	159345973
chrX	156040895
chr8	145138636
chr9	138394717
chr11	135086622
chr10	133797422
chr12	133275309
chr13	114364328
chr14	107043718
chr15	101991189
chr16	90338345
chr17	83257441
chr18	80373285
chr20	64444167
chr19	58617616
chrY	57227415
chr22	50818468
chr21	46709983
chr15_KI270905v1_alt	5161414
chr6_GL000256v2_alt	4929269
chr6_GL000254v2_alt	4827813
chr6_GL000251v2_alt	4795265
chr6_GL000253v2_alt	4677643
chr6_GL000250v2_alt	4672374
chr6_GL000255v2_alt	4606388
chr6_GL000252v2_alt	4604811
chr17_KI270857v1_alt	2877074
chr16_KI270853v1_alt	2659700
chr16_KI270728v1_random	1872759
chr17_GL000258v2_alt	1821992
chr5_GL339449v2_alt	1612928
chr14_KI270847v1_alt	1511111
chr17_KI270908v1_alt	1423190
chr14_KI270846v1_alt	1351393
chr5_KI270897v1_alt	1144418
chr7_KI270803v1_alt	1111570
chr19_GL949749v2_alt	1091841
chr19_KI270938v1_alt	1066800
chr19_GL949750v2_alt	1066390
chr19_GL949748v2_alt	1064304
chr1

**RUN**

In [6]:
ls -d ${FD_RES}/data/Input?_20x/

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input1_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input2_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input3_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input4_20x/
/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/data/Input5_20x/


In [9]:
ls ${FD_RES}/data/Input1_20x/

chr10.bed  chr14.bed  chr18.bed  chr21.bed  chr4.bed  chr8.bed
chr11.bed  chr15.bed  chr19.bed  chr22.bed  chr5.bed  chr9.bed
chr12.bed  chr16.bed  chr1.bed   chr2.bed   chr6.bed  chrX.bed
chr13.bed  chr17.bed  chr20.bed  chr3.bed   chr7.bed  chrY.bed


In [11]:
ls -d ${FD_LOG}
echo ${NODE}

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log
all


In [14]:
source ../config_duke.sh
source ../config_load_module_bedtools.sh

sbatch -p ${NODE} \
    --array 0-23 \
    --mem 8G \
    -o ${FD_LOG}/coverage_chrom_input_20x.%a.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
CHROMS=($(seq 1 22) X Y)
CHROM=chr${CHROMS[${SLURM_ARRAY_TASK_ID}]}
TARGET=${CHROM}

### set input and output
FD_BEDS=($(ls -d ${FD_RES}/data/Input?_20x/))
FN_BED=${CHROM}.bed
FD_OUT=${FD_RES}/coverage

FD_GEN=${FD_ANN}/genome/hg38
FN_GEN=hg38.chrom.sizes

### print start message
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}

### loop through each sample and count each fragment of a chromosome
echo "RUN: loop through samples; count fragment"

for FD_BED in ${FD_BEDS[@]}; do
    SAMPLE=$(basename ${FD_BED})
    FD_OUT=${FD_RES}/coverage/${SAMPLE}
    FN_OUT=${TARGET}.bedgraph
    
    ### START Message
    echo ++++++++++++++++++++++++++++++++++++++++++++++++
    echo "Input  file: " ${FD_BED}/${FN_BED}
    echo "Output file: " ${FD_OUT}/${FN_OUT}
    echo
    echo "Show the first few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    head -n 3 ${FD_BED}/${FN_BED}
    echo
    echo "Show the last few lines of the input file"
    echo ${FD_BED}/${FN_BED}
    tail -n 3 ${FD_BED}/${FN_BED}
    echo
    
    ### init: create output folder if not exist
    mkdir -p ${FD_OUT}
    
    ### Bedtools genomecov to get bedgraph
    echo "Get the bedgraph"
    bedtools genomecov -i ${FD_BED}/${FN_BED} -g ${FD_GEN}/${FN_GEN} -bg > ${FD_OUT}/${FN_OUT}
    echo
    
    ### END Message
    echo "Show the first few lines of the output file"
    echo ${FD_OUT}/${FN_OUT}
    head -n 3 ${FD_OUT}/${FN_OUT}
    echo
    echo "Show the last few lines of the output file"
    echo ${FD_OUT}/${FN_OUT}
    tail -n 3 ${FD_OUT}/${FN_OUT}
    echo
done
echo "Done!"

EOF

Submitted batch job 26142624


**CHECK**

In [15]:
readlink -f ${FD_LOG}/coverage_chrom_input_20x.0.txt

/gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect/log/coverage_chrom_input_20x.0.txt


In [3]:
cat ${FD_LOG}/coverage_chrom.16.txt

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation

Slurm Array Index:  16
RUN: loop through samples; count fragment
++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /work/kk319/out/CombEffect_STARR/data/Input//chr17.bed
Output file:  /work/kk319/out/CombEffect_STARR/coverage/Input/chr17.bedgraph

Show the first few lines of the input file
/work/kk319/out/CombEffect_STARR/data/Input//chr17.bed
chr17	159510	160362
chr17	182115	183108
chr17	197679	198597
chr17	201123	201925
chr17	201133	201985

Show the last few lines of the input file
/work/kk319/out/CombEffect_STARR/data/Input//chr17.bed
chr17	83236101	83237202
chr17	83240987	83242875
chr17	83241073	83242992
chr17	83241856	83242863
chr17	83244716	83245595

Get the bedgraph

Show the first 

In [14]:
cat ${FD_LOG}/coverage_chrom.17.txt

You are on Duke Server: DCC
BASE DIRECTORY:     /work/kk319
WORK DIRECTORY:     /work/kk319/out/CombEffect_STARR
LOG  DIRECTORY:     /work/kk319/out/CombEffect_STARR/log
PATH OF SOURCE:     /work/kk319/source
PATH OF EXECUTABLE: /work/kk319/exe
PATH OF ANNOTATION: /work/kk319/annotation

Slurm Array Index:  17
RUN: loop through samples; count fragment
++++++++++++++++++++++++++++++++++++++++++++++++
Input  file:  /work/kk319/out/CombEffect_STARR/data/Input//chr18.bed
Output file:  /work/kk319/out/CombEffect_STARR/coverage/Input/chr18.bedgraph

Show the first few lines of the input file
/work/kk319/out/CombEffect_STARR/data/Input//chr18.bed
chr18	21483	22446
chr18	21483	22446
chr18	28214	29137
chr18	37749	38649
chr18	38694	39604

Show the last few lines of the input file
/work/kk319/out/CombEffect_STARR/data/Input//chr18.bed
chr18	80260746	80261661
chr18	80260962	80262023
chr18	80261109	80262066
chr18	80261421	80262495
chr18	80261492	80262651

Get the bedgraph

Show the first few lines 